In [3]:
!pip install -q chromadb sentence-transformers transformers accelerate scikit-learn pandas numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.7/20.7 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.0 MB/s eta 

In [4]:
!pip install langgraph langchain langchain-community

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sou

In [6]:
import os, random, time
from datetime import datetime, UTC
import numpy as np
import pandas as pd
from typing import List, Dict
from langgraph.graph import StateGraph, END
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

In [25]:
CHROMA_DIR = "./chromadb_langgraph"
EMBED_MODEL = "all-MiniLM-L6-v2"
LLM_MODEL = "google/flan-t5-small"
ALERT_THRESHOLD = 0.75
SIM_STEPS = 60
os.makedirs(CHROMA_DIR, exist_ok=True)

In [9]:
embeddings = SentenceTransformerEmbeddings(model_name=EMBED_MODEL)
vector_store = Chroma(collection_name="memory", embedding_function=embeddings, persist_directory=CHROMA_DIR)

/tmp/ipython-input-2785855123.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name=EMBED_MODEL)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipython-input-2785855123.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(collection_name="memory", embedding_function=embeddings, persist_directory=CHROMA_DIR)


In [10]:
llm_tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL)
llm_model = AutoModelForSeq2SeqLM.from_pretrained(LLM_MODEL)
llm_pipe = pipeline("text2text-generation", model=llm_model, tokenizer=llm_tokenizer, device=-1)

def call_local_llm(prompt: str, max_length: int = 256):
    out = llm_pipe(prompt, max_length=max_length, do_sample=False)
    return out[0]["generated_text"].strip()

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu


In [11]:
state = {
    "transactions": [],
    "ticks": [],
    "fraud_alerts": [],
    "blocked_accounts": {},
    "invest_ranking": []
}

In [12]:
fraud_model = None
fraud_scaler = None
fraud_feat = None
inv_model = None
inv_scaler = None
inv_feat = None

In [13]:
def generate_transactions(n=1):
    merchants = ["grocery","electronics","crypto","travel","utilities","gambling"]
    for i in range(n):
        f = random.random() < 0.08
        amt = np.random.exponential(80) + (1000 if f else 0)
        tx = {
            "id": f"tx_{len(state['transactions'])+i}",
            "acct": random.randint(1,2000),
            "amount": float(amt),
            "merchant": random.choice(merchants),
            "country": random.choice(["US","IN","DE","FR","CA"]),
            "label": int(f),
            "timestamp": datetime.now(UTC).isoformat(),
            "features": np.array([amt/3000.0, random.random(), 1.0 if f else 0.0, random.random(), random.random()])
        }
        yield tx

In [14]:
def generate_ticks(n=1):
    syms = ["AAPL", "TSLA", "BTC-USD", "GOOG"]
    prices = {sym: 100 + random.random() * 50 for sym in syms}
    for i in range(n):
        sym = random.choice(syms)
        change = np.random.normal(0, 0.02)
        prices[sym] *= (1 + change)
        price = prices[sym]
        vol = int(np.random.exponential(100))
        label = int(change > 0)
        yield {
            "id": f"tick_{len(state['ticks'])+i}",
            "symbol": sym,
            "price": float(price),
            "volume": vol,
            "change": change,
            "label": label,
            "features": np.array([price/200.0, vol/500.0, change, random.random(), random.random()])
        }


In [15]:
def index_texts(ids, texts):
    vector_store.add_texts(texts, ids=ids)

def retrieve_texts(query: str, k: int = 5):
    docs = vector_store.similarity_search(query, k=k)
    return [d.page_content for d in docs]

In [16]:
def train_fraud(batch):
    global fraud_model, fraud_scaler, fraud_feat
    if not batch:
        return []
    X = np.vstack([t["features"] for t in batch])
    y = np.array([t["label"] for t in batch])
    if (fraud_model is None) or (fraud_feat != X.shape[1]):
        fraud_model = SGDClassifier(loss="log_loss", max_iter=5, tol=None, learning_rate="adaptive", eta0=0.01)
        fraud_scaler = StandardScaler()
        fraud_scaler.fit(X)
        fraud_model.partial_fit(fraud_scaler.transform(X), y, classes=[0, 1])
        fraud_feat = X.shape[1]
        print(f"[INFO] Fraud model initialized ({fraud_feat} features).")
    else:
        fraud_model.partial_fit(fraud_scaler.transform(X), y)
    probs = fraud_model.predict_proba(fraud_scaler.transform(X))[:, 1]
    alerts = []
    for t, p in zip(batch, probs):
        if p > ALERT_THRESHOLD:
            alerts.append({
                "tx_id": t["id"],
                "acct": t["acct"],
                "prob": round(float(p), 3),
                "amount": t["amount"],
                "merchant": t["merchant"],
                "country": t["country"],
                "timestamp": t["timestamp"]
            })
    if alerts:
        print(f"###### {len(alerts)} fraud alerts detected")
        for a in alerts:
            print(f"  TX={a['tx_id']} ACC={a['acct']} PROB={a['prob']} AMT={a['amount']}")
        state["fraud_alerts"].extend(alerts)
    return alerts

In [17]:
def train_invest(batch):
    global inv_model, inv_scaler, inv_feat
    if not batch: return []
    X = np.vstack([t["features"] for t in batch])
    y = np.array([t["label"] for t in batch])
    if (inv_model is None) or (inv_feat != X.shape[1]):
        inv_model = SGDClassifier(loss="log_loss", max_iter=5, tol=None)
        inv_scaler = StandardScaler()
        inv_scaler.fit(X)
        inv_model.partial_fit(inv_scaler.transform(X), y, classes=[0,1])
        inv_feat = X.shape[1]
    else:
        inv_model.partial_fit(inv_scaler.transform(X), y)
    probs = inv_model.predict_proba(inv_scaler.transform(X))[:,1]
    df = pd.DataFrame({"symbol":[t["symbol"] for t in batch],"prob":probs})
    ranking = df.groupby("symbol")["prob"].mean().sort_values(ascending=False)
    state["invest_ranking"] = list(ranking.items())
    return state["invest_ranking"]

In [18]:
def update_blocked(alerts):
    for a in alerts:
        acc = a["acct"]
        info = state["blocked_accounts"].get(acc, {"count":0,"blocked":False})
        info["count"] += 1
        if info["count"] >= 3:
            info["blocked"] = True
        state["blocked_accounts"][acc] = info

def get_blocked():
    return [k for k,v in state["blocked_accounts"].items() if v.get("blocked")]

In [19]:
def get_fraud_summary(n=5):
    df = pd.DataFrame(state["fraud_alerts"])
    if df.empty: return []
    return df.sort_values("prob", ascending=False).head(n).to_dict("records")

def get_investment_picks(n=3):
    return state["invest_ranking"][:n]

In [20]:
def synthesize_answer(query: str, retrieved, tool_outputs):
    prompt = f"""
    You are **FinGuard**, an AI financial intelligence assistant.
    Your task is to provide **clear, factual, and risk-aware** insights
    based on both structured model outputs and retrieved historical context.
    User query: {query}
    Structured data:{tool_outputs}
    Retrieved context:{retrieved[:3]}
    Explain clearly and concisely."""
    return call_local_llm(prompt)

In [21]:
def classify_intent(s):
    q = s["query"].lower()
    if "fraud" in q: s["intent"] = "fraud"
    elif "invest" in q: s["intent"] = "invest"
    elif "block" in q: s["intent"] = "block"
    else: s["intent"] = "general"
    return s

In [ ]:
def retrieve_node(s):
    s["retrieved"] = retrieve_texts(s["query"], k=5)
    return s

In [ ]:
def tool_node(s):
    intent = s.get("intent")
    tools = {}
    if intent == "fraud": tools["fraud_summary"] = get_fraud_summary()
    if intent == "invest": tools["invest_picks"] = get_investment_picks()
    if intent == "block":
        tools["blocked_accounts"] = [{"account": acc, "details": state["blocked_accounts"][acc]} for acc in get_blocked()]
    s["tool_outputs"] = tools
    return s

In [ ]:
def llm_reason_node(s):
    s["answer"] = synthesize_answer(s["query"], s.get("retrieved", []), s.get("tool_outputs", {}))
    return s

In [22]:
from typing import TypedDict, Any
class AgentState(TypedDict):
    query: str
    intent: str
    retrieved: list
    tool_outputs: dict
    answer: str
graph = StateGraph(AgentState)
graph.add_node("ClassifyIntent", classify_intent)
graph.add_node("RetrieveRAG", retrieve_node)
graph.add_node("RunTools", tool_node)
graph.add_node("LLMReason", llm_reason_node)
graph.add_edge("ClassifyIntent", "RetrieveRAG")
graph.add_edge("RetrieveRAG", "RunTools")
graph.add_edge("RunTools", "LLMReason")
graph.add_edge("LLMReason", END)
graph.set_entry_point("ClassifyIntent")
agent_graph = graph.compile()


In [23]:
def run_simulation(steps=SIM_STEPS):
    for step in range(steps):
        tx_batch = list(generate_transactions(random.randint(10,30)))
        tick_batch = list(generate_ticks(random.randint(5,15)))
        state["transactions"].extend(tx_batch)
        state["ticks"].extend(tick_batch)
        index_texts([t["id"] for t in tx_batch], [f"Transaction {t['id']} amount={t['amount']} merchant={t['merchant']} label={t['label']}" for t in tx_batch])
        index_texts([t["id"] for t in tick_batch], [f"Tick {t['id']} symbol={t['symbol']} price={t['price']}" for t in tick_batch])
        alerts = train_fraud(tx_batch)
        update_blocked(alerts)
        train_invest(tick_batch)
        if step % 10 == 0:
            print(f"[STEP {step}] TX={len(state['transactions'])} FRAUD={len(state['fraud_alerts'])} BLOCKED={len(get_blocked())}")
        time.sleep(0.02)

In [26]:
run_simulation(10)

queries = [
    "Show me recent fraud alerts and risky accounts.",
    "Which accounts are currently blocked?",
    "Give me the top investment picks right now."
]
for q in queries:
    print(f"\n> Query: {q}")
    result = agent_graph.invoke({"query": q})
    print("🧠 Agentic Answer:\n", result["answer"])

###### 1 fraud alerts detected
  TX=tx_228 ACC=551 PROB=0.816 AMT=1141.0987346024021
[STEP 0] TX=234 FRAUD=24 BLOCKED=0
###### 1 fraud alerts detected
  TX=tx_238 ACC=119 PROB=0.809 AMT=1014.2626862018261
###### 2 fraud alerts detected
  TX=tx_294 ACC=1071 PROB=0.855 AMT=1110.0448072004258
  TX=tx_295 ACC=810 PROB=0.823 AMT=1032.3748219573597
###### 2 fraud alerts detected
  TX=tx_346 ACC=30 PROB=0.82 AMT=1049.9493412286738
  TX=tx_350 ACC=882 PROB=0.833 AMT=1073.7511268605608
###### 2 fraud alerts detected
  TX=tx_357 ACC=1409 PROB=0.84 AMT=1001.9579302797335
  TX=tx_365 ACC=448 PROB=0.848 AMT=1016.1465026764704
###### 1 fraud alerts detected
  TX=tx_369 ACC=1475 PROB=0.88 AMT=1123.8422312865348

> Query: Show me recent fraud alerts and risky accounts.


Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


🧠 Agentic Answer:
 tx_id, tx_id, tx_365, acct, 1071, prob, 0.848, amount, 1016.1465026764704, merchant: grocery, country, DE, timestamp: 2025-10-23T11:21:43.464916+00:00, tx_id, tx_357, acct, 448, prob, 0.848, amount, 1001.9579302797335, merchant: crypto, country, US, timestamp: 2025-10-23T11:21:43.033393+00:00, tx_id, tx_357, acct, 448, prob, 0.848, amount, 1001.9579302797335, merchant: crypto, country, US, timestamp: 2025-10-23T11:21:43.033393+00:00, tx_id, tx_357, acct,

> Query: Which accounts are currently blocked?
🧠 Agentic Answer:
 Transaction tx_334 amount=130.7230824381077 merchant=crypto label=0; Transaction tx_180 amount=163.1367318786168 merchant=crypto label=0; Transaction tx_172 amount=226.5031994119785 merchant=gambling label=0

> Query: Give me the top investment picks right now.


Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


🧠 Agentic Answer:
 Transaction tx_373 amount=166.7230796755393 merchant=gambling label=0; Transaction tx_253 amount=84.79569063326252 merchant=gambling label=0; Transaction tx_253 amount=84.79569063326252 merchant=gambling label=0


In [28]:
!jupyter nbconvert --ClearMetadataPreprocessor.enabled=True --ClearOutputPreprocessor.enabled=True --to notebook --inplace "pathway (1).ipynb"


[NbConvertApp] WARNING | pattern 'pathway (1).ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
   